<a href="https://colab.research.google.com/github/Shubham04689/colab_notebooks/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data = [{'message':"i like the movie","sentiment":"pisitive"},
        {'message':"movie is not good","sentiment":"negative"},
        {'message':"wasted my time","sentiment":"negative"},
        {'message':"liked the movie","sentiment":"pisitive"},
        ]

In [2]:
df = pd.DataFrame(data)

In [3]:
df.head()

,message,sentiment
0,i like the movie,pisitive
1,movie is not good,negative
2,wasted my time,negative
3,liked the movie,pisitive


In [4]:
messages = df['message']
labels  = df['sentiment']

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(messages)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),columns= vectorizer.get_feature_names_out())
tfidf_df

,good,is,like,liked,movie,my,not,the,time,wasted
0,0.000000,0.000000,0.702035,0.000000,0.448100,0.00000,0.000000,0.553492,0.00000,0.00000
1,0.541736,0.541736,0.000000,0.000000,0.345783,0.00000,0.541736,0.000000,0.00000,0.00000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000,0.000000,0.57735,0.57735
3,0.000000,0.000000,0.000000,0.702035,0.448100,0.00000,0.000000,0.553492,0.00000,0.00000


In [7]:
import math
TF_IDF_movie = (1/4) * math.log(4/3)
TF_IDF_movie

0.07192051811294521

In [8]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel='linear')

In [9]:
svm_classifier.fit(tfidf_matrix,labels)

SVC(kernel='linear')

In [10]:
def predict_sentiment(texts):
  tfidf_texts = vectorizer.transform(texts)
  predictions = svm_classifier.predict(tfidf_texts)
  return predictions

In [11]:
sample = ['movie is not good']
pred_result = predict_sentiment(sample)
pred_result[0]

'negative'

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(messages)
count_df = pd.DataFrame(count_matrix.toarray(),columns = count_vectorizer.get_feature_names_out())
count_df.head()

,good,is,like,liked,movie,my,not,the,time,wasted
0,0,0,1,0,1,0,0,1,0,0
1,1,1,0,0,1,0,1,0,0,0
2,0,0,0,0,0,1,0,0,1,1
3,0,0,0,1,1,0,0,1,0,0


In [13]:
svm_classifier_count = SVC(kernel='linear')
svm_classifier_count.fit(count_matrix,labels)

SVC(kernel='linear')

In [14]:
def predict_sentiment_count(texts):
  count_texts = count_vectorizer.transform(texts)
  predictions = svm_classifier_count.predict(count_texts)
  return predictions

In [15]:
sample = ['movie is not good']
pred_result = predict_sentiment_count(sample)
pred_result[0]

'negative'

In [16]:
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.preprocessing import scale
import numpy as np

In [17]:
word2vec_model = api.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [18]:
def feature_vector(text,model):
  words = text.split()
  feature_vec = np.zeros((model.vector_size,),dtype='float32')
  nwords = 0
  for word in words:
    if word in model.key_to_index:
      nwords+=1
      feature_vec = np.add(feature_vec,model[word])
  if nwords>0:
    feature_vec = np.divide(feature_vec,nwords)
  return feature_vec

In [19]:
w2vec_features = np.array([feature_vector(text,word2vec_model) for text in messages])

In [20]:
w2vec_features

array([[ 7.26924837e-03,  3.35020006e-01,  6.49890006e-01,
        -5.26047528e-01, -1.43585518e-01,  1.86641499e-01,
        -3.68413746e-01,  9.62462500e-02, -1.77842498e-01,
        -1.56282753e-01,  2.20807493e-01, -1.52342498e-01,
         1.69697493e-01,  3.38559970e-02,  8.08584988e-02,
        -3.23575139e-02,  2.32260004e-01,  3.02332491e-01,
        -2.15605021e-01,  6.12187505e-01,  3.54585499e-01,
         2.96474993e-01, -1.86075270e-03, -2.58514762e-01,
         5.08992553e-01,  2.46700004e-01, -1.42585248e-01,
        -5.43537438e-01,  4.26375121e-02, -3.06697488e-01,
         2.85595022e-02,  5.00990033e-01,  2.59411514e-01,
         5.75082488e-02,  2.18327507e-01,  3.63980740e-01,
         6.65675104e-02,  3.67342740e-01,  2.74040014e-01,
        -4.01309490e-01, -2.16328755e-01, -5.90324998e-02,
        -1.24485001e-01, -3.38887513e-01, -1.16319984e-01,
         8.36025029e-02,  9.54199880e-02, -4.14161503e-01,
        -1.85005367e-04, -8.05810034e-01, -1.44639254e-0

In [21]:
w2v_features_scaled = scale(w2vec_features)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:261: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:280: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [22]:
svm_w2 = SVC(kernel='linear')
svm_w2.fit(w2v_features_scaled,labels)

SVC(kernel='linear')

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
documents = ['king','man']

In [25]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)


In [26]:
vector_king  = X.toarray()[0]
vector_man = X.toarray()[1]
vector_king,vector_man

(array([1, 0]), array([0, 1]))

In [27]:
dot_product = sum(a*b  for a,b in zip(vector_king,vector_man))
magnitude_king = sum(a*a for a in vector_king) ** 0.5
magnitude_man = sum(b*b for b in vector_man) ** 0.5
print(dot_product/(magnitude_king*magnitude_man))

0.0


In [28]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [29]:
import spacy
nlp = spacy.load('en_core_web_md')

In [30]:
vector_king = nlp("king").vector
vector_man = nlp("man").vector
dot_product = sum(a*b  for a,b in zip(vector_king,vector_man))
magnitude_king = sum(a*a for a in vector_king) ** 0.5
magnitude_man = sum(b*b for b in vector_man) ** 0.5
print(dot_product/(magnitude_king*magnitude_man))

0.41661588732100174


In [31]:
!pip install textdistance

In [32]:
import textdistance as td

In [33]:
td.hamming('book','look')

1

In [34]:
td.hamming.normalized_similarity('book','look')

0.75

In [35]:
(4-1)/4

0.75

In [36]:
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [37]:
result_vector = model['king']-model['man'] +model['woman']
closest_word = model.similar_by_vector(result_vector,topn=10)
closest_words = [word for word in closest_word if word[0] not in ['king','man','woman']]
print(closest_words[0])

('queen', 0.7300517559051514)
